In [1]:
#-*- coding: utf-8 -*-

from multiprocessing import Process

from pandas import DataFrame
from pymongo import MongoClient
from firebase.firebase import FirebaseApplication

In [2]:
connection = MongoClient('localhost', 27018)
connection.database_names()

['admin', 'local', 'portal_transparencia']

In [3]:
db = connection['portal_transparencia']
db.collection_names()

['dados_abertos']

In [4]:
collection = db['dados_abertos']
collection.count()

38994

In [5]:
cursor = collection.find({})

In [6]:
df = DataFrame(list(cursor))

In [27]:
# df_2017 = df[(df.ano.apply(int) == 2017)][['estado', 'municipio', 'acoes']]
# df_2017.estado.drop_duplicates()

In [7]:
anos = df['ano'].drop_duplicates()
estados = df['estado'].drop_duplicates()

In [8]:
anos = sorted(['2017','2016','2015'])
estados = sorted(estados)

estados.remove('TRANSFERÊNCIAS A ORGANISMOS MULTIGOVERNAMENTAIS')
estados.remove('TRANSFERÊNCIAS AO EXTERIOR')

In [9]:
items = {}
items['BRASIL'] = {}
dict_items = {
    'NORTE': ['ACRE', 'AMAZONAS', 'AMAPÁ', 'PARÁ', 'RONDÔNIA', 'RORAIMA', 'TOCANTINS'],
    'NORDESTE': ['ALAGOAS', 'BAHIA', 'CEARÁ', 'MARANHÃO', 'PARAÍBA','PERNAMBUCO',
                 'PIAUÍ', 'RIO GRANDE DO NORTE', 'SERGIPE'],
    'CENTRO-OESTE': ['GOIÁS', 'MATO GROSSO DO SUL', 'MATO GROSSO'],
    'SUDESTE': ['ESPÍRITO SANTO','MINAS GERAIS','RIO DE JANEIRO','SÃO PAULO'],
    'SUL': ['PARANÁ','RIO GRANDE DO SUL','SANTA CATARINA']
}

regioes = ['NORTE','NORDESTE', 'CENTRO-OESTE', 'SUDESTE', 'SUL']

all_datas = {}

_item = {}
for k in regioes:
    _item[k] = {}


for estado in estados:
    for k in regioes:
        if estado in dict_items[k]:
            _item[k][estado] = {}
            
            
for estado in estados:
    for k in regioes:
        if estado in dict_items[k]:
            for ano in anos:
                _item[k][estado][ano] = {}
    
for estado in estados:
    for k in regioes:
        if estado in dict_items[k]:
            for ano in anos:
                _municipios = sorted(df[(df.estado == estado) & (df.ano == ano)]['municipio'].drop_duplicates())
                for municipio in _municipios:
                    link_datas = "-".join(['BRASIL', k, estado, ano, municipio])
                    _item[k][estado][ano][municipio] = link_datas
                    all_datas[link_datas] = {}
                    all_datas[link_datas]['DESPESAS'] = list(
                        df[(df.estado == estado) & (df.municipio == municipio) & (df.ano == ano)]['acoes']
                    ).pop()
                    
                items['BRASIL'].update(_item)

# DB = {'estados': items}
DB = items

In [10]:
DB

{'BRASIL': {'CENTRO-OESTE': {'GOIÁS': {'2015': {'ABADIA DE GOIAS': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-ABADIA DE GOIAS',
     'ABADIANIA': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-ABADIANIA',
     'ACREUNA': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-ACREUNA',
     'ADELANDIA': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-ADELANDIA',
     'AGUA FRIA DE GOIAS': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-AGUA FRIA DE GOIAS',
     'AGUA LIMPA': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-AGUA LIMPA',
     'AGUAS LINDAS DE GOIAS': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-AGUAS LINDAS DE GOIAS',
     'ALEXANIA': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-ALEXANIA',
     'ALOANDIA': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-ALOANDIA',
     'ALTO HORIZONTE': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-ALTO HORIZONTE',
     'ALTO PARAISO DE GOIAS': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-ALTO PARAISO DE GOIAS',
     'ALVORADA DO NORTE': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-ALVORADA DO NORTE',
     'AMARALINA': 'BRASIL-CENTRO-OESTE-GOIÁS-2015-AMARALINA',
     'AMERICANO DO BRASIL': 'BRASIL-CENTRO-OESTE-GOIÁ

In [11]:
# Firebase Integration

DSN = 'https://apptransparenciaeducativa.firebaseio.com/'

authentication = None
firebase = FirebaseApplication(DSN, authentication)

In [13]:
firebase.patch('/', all_datas)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [18]:
def firebase_patch(lista):
    # Firebase Integration
    DSN = 'https://apptransparenciaeducativa.firebaseio.com/'
    authentication = None
    firebase = FirebaseApplication(DSN, authentication)
    firebase.patch(lista[0], lista[1])

In [23]:
OLD_DB = DB.copy()
despesas = "DESPESAS"
for pais in ["BRASIL"]:
    for regiao in regioes:
        estados = dict_items[regiao]
        for estado in estados:
            for ano in anos:
                processos = []
                municipios = OLD_DB[pais][regiao][estado][ano][despesas]
                for municipio in municipios:                   
                    list_acoes = list(
                        df[(df.estado == estado) & (df.municipio == municipio) & (df.ano == ano)]['acoes']
                    ).pop()

                    _lista = ["/" + ("/".join([pais,regiao,estado,ano,despesas])), {'municipio':list_acoes}]
                    processos.append(Process(target=firebase_patch, args=(_lista,)))
                
                while processos:
                    p = processos.pop()
                    p.start()
                    p.join()
                            

KeyboardInterrupt: 